In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms

from PIL import Image

import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import re
import os
import numpy as np

import math

In [ ]:
!pip install torch
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!unzip "/content/drive/MyDrive/Driver Drowsiness Dataset (DDD).zip" -d "/content/drive/MyDrive/newdata/"

Archive:  /content/drive/MyDrive/Driver Drowsiness Dataset (DDD).zip
replace /content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Drowsy/A0001.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Drowsy/A0002.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Add image paths, labels, file names and person IDs into lists
image_path, label, file_name, person_ID = [], [], [], []

pattern = re.compile(r'^[a-zA-Z][a-zA-Z]?') #identification criteria of participant

data_dir = "/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/"

for class_name in os.listdir(data_dir):  #goes through all folders in DDD folder
  for path in os.listdir(data_dir+class_name):  #goes through all files in both folders and updates file path and labels
    if class_name == 'Drowsy':
      label.append(0)
    else:
      label.append(1)
    image_path.append(os.path.join(data_dir, class_name, path))
    file_name.append(path)

    person_ID.append(pattern.findall(path)[0])  #identification of participant


#dataframe with path, label, file name, person_ID of all Persons
df = pd.DataFrame()
df['images'] = image_path
df['label'] = label
df['name'] = file_name
df['person'] = person_ID

print('df')
print(df.shape)


#add Person E and G only to the Test Dataset
dfTest = pd.concat([df[df['person']=='e'], df[df['person']=='g'], df[df['person']=='E'], df[df['person']=='G']]) #, df[df['person']=='S'], df[df['person']=='s']])
print('dfTest')
print(dfTest.shape)


#remove Person E and G from the dataset with all Persons
dfWithout = df
dfWithout = dfWithout.drop(index=(df[df['person']=='e'].index))
dfWithout = dfWithout.drop(index=(df[df['person']=='E'].index))
dfWithout = dfWithout.drop(index=(df[df['person']=='g'].index))
dfWithout = dfWithout.drop(index=(df[df['person']=='G'].index))

#dfWithout = dfWithout.drop(index=(df[df['person']=='s'].index))
#dfWithout = dfWithout.drop(index=(df[df['person']=='S'].index))

print('dfWithout')
print(dfWithout.shape)


#converting imagepath and label of takeout persons G, E into an nparray
imagePathTakeout = dfTest['images']
labelTakeout = dfTest['label']

imagePathTO = np.array(imagePathTakeout).reshape([-1,1])
labelTO = np.array(labelTakeout).reshape([-1,1])
testTO = np.hstack((imagePathTO, labelTO))

print('testTO')
print(testTO.shape)


#converting imagepath and label of all other persons into an nparray
imagePathWithout = dfWithout['images']
labelWithout = dfWithout['label']

image_path = np.array(imagePathWithout).reshape([-1,1])
label = np.array(labelWithout).reshape([-1,1])
comp_data = np.hstack((image_path, label))


df
(41793, 4)
dfTest
(2570, 4)
dfWithout
(39223, 4)
testTO
(2570, 2)


In [ ]:
print(testTO)

[['/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/e0382.png'
  1]
 ['/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/e0399.png'
  1]
 ['/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/e0415.png'
  1]
 ...
 ['/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Drowsy/G0534.png'
  0]
 ['/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Drowsy/G0529.png'
  0]
 ['/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Drowsy/G0525.png'
  0]]


In [ ]:
df.groupby("person").size()

person
A     1411
B      315
C      335
D      179
E      962
F      415
G      499
H      508
I     1095
J      474
K      630
L      732
M      777
N     1156
O     1097
P      963
Q      562
R      204
S      487
T      933
U      420
V      653
W     1162
X     1749
Y     1112
ZA     621
ZB    1551
ZC    1346
a     1252
b      409
c      400
d     1005
e     1000
g      109
h      571
i     1045
j      717
k      538
l      381
m      473
n      957
o      671
p      190
q      521
r      522
s      457
u      510
v     1002
w      493
x     1143
y     1500
za    1054
zb    1237
zc    1288
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

#nparray with all other persons (except of E, G) gets split into train and test dataset
trainW, testW = train_test_split(comp_data, test_size=0.2, random_state=1234)
print('trainW')
print(trainW.shape)

print('testW')
print(testW.shape)


#train dataset
train = trainW


#test dataset is a combination of the takeout of person E, G and the part that was split to test via train_test_split
test = np.concatenate((testTO, testW))


print('test')
print(test.shape)
print(train)

trainW
(31378, 2)
testW
(7845, 2)
test
(10415, 2)
[['/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/zb0371.png'
  1]
 ['/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Drowsy/P0931.png'
  0]
 ['/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Drowsy/ZB1536.png'
  0]
 ...
 ['/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Drowsy/X0938.png'
  0]
 ['/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Drowsy/L0278.png'
  0]
 ['/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Drowsy/S0144.png'
  0]]


In [ ]:
class DDDdataset(Dataset):
  def __init__(self, image_paths, image_labels, transform):
    super().__init__()
    self.paths = image_paths
    self.labels = image_labels
    self.len = len(self.paths)
    self.transform = transform
  
  def __len__(self): return self.len
  
  def __getitem__(self, index):
    path = self.paths[index]
    label = self.labels[index]
    target_tensor=torch.from_numpy(np.array(label,dtype=np.int16))
    #image = Image.open(path).convert('RGB')
    image = cv2.imread(path,cv2.IMREAD_UNCHANGED)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = self.transform(image)
    return (image, target_tensor, path)

In [ ]:
#transformation for the calculation of mean and std for the train dataset
transformationCalc = transforms.Compose([
    #ToTensor() includes an automatic scaling from the interval (0,255) to (0.0,1.0)
    transforms.ToTensor()
])

#train dataset and loader for the calculation of mean and std for the train dataset
trainDatasetCalc = DDDdataset(train[:,0], train[:,-1], transformationCalc)

loaderCalc = DataLoader(trainDatasetCalc, batch_size=128, shuffle=True)

In [ ]:
numberOfPixelsPerImage = 227*227
noOfImages = train.shape[0]


#lists to collect the sum of all pixel values in the batch for R, G, B
sumOfBatchesR, sumOfBatchesG, sumOfBatchesB = [], [], []
squaredSumOfBatchesR, squaredSumOfBatchesG, squaredSumOfBatchesB = [], [], []


#in calcBatch(es) with 128 pictures each perform sum operations
for calcBatch in loaderCalc:
  #calcBatch[0] with dimension (128, 227, 227, 3)
  currentBatch = calcBatch[0]
  #currentBatch = currentBatch.float()


  rgbSumOfBatch = currentBatch.sum(axis=2) #reduce column of one picture to one column (128, 227, 3)
  rgbSumOfBatch = rgbSumOfBatch.sum(axis=1) #reduce row of one picture to one row (128, 3)
  rgbSumOfBatch = rgbSumOfBatch.sum(axis=0) #combine all pictures of the batch to 3 values for rgb (3)
  
  #sum of the batch for R, G, B is stored for final calculation
  sumOfBatchesR.append(rgbSumOfBatch[0])
  sumOfBatchesG.append(rgbSumOfBatch[1])
  sumOfBatchesB.append(rgbSumOfBatch[2])


  rgbSquaredSumOfBatch = ((currentBatch)**2).sum(axis=2) #reduce column of one picture to one column (128, 227, 3)
  rgbSquaredSumOfBatch = rgbSquaredSumOfBatch.sum(axis=1) #reduce row of one picture to one row (128, 3)
  rgbSquaredSumOfBatch = rgbSquaredSumOfBatch.sum(axis=0) #combine all pictures of the batch to 3 values for rgb (3)

  #squared sum of the batch for R, G, B is stored for final calculation
  squaredSumOfBatchesR.append(rgbSquaredSumOfBatch[0])
  squaredSumOfBatchesG.append(rgbSquaredSumOfBatch[1])
  squaredSumOfBatchesB.append(rgbSquaredSumOfBatch[2])


#final sum for R, G, B
sumOfBatchesR = np.array(sumOfBatchesR).sum()
sumOfBatchesG = np.array(sumOfBatchesG).sum()
sumOfBatchesB = np.array(sumOfBatchesB).sum()

#calculation of the mean for R, G, B
meanR = sumOfBatchesR/(numberOfPixelsPerImage*noOfImages)
meanG = sumOfBatchesG/(numberOfPixelsPerImage*noOfImages)
meanB = sumOfBatchesB/(numberOfPixelsPerImage*noOfImages)

#final squared sum for R, G, B
squaredSumOfBatchesR = np.array(squaredSumOfBatchesR).sum()
squaredSumOfBatchesG = np.array(squaredSumOfBatchesG).sum()
squaredSumOfBatchesB = np.array(squaredSumOfBatchesB).sum()

#final variance
varianceR = (squaredSumOfBatchesR - ((sumOfBatchesR)**2)/(numberOfPixelsPerImage*noOfImages))/(numberOfPixelsPerImage*noOfImages)
varianceG = (squaredSumOfBatchesG - ((sumOfBatchesG)**2)/(numberOfPixelsPerImage*noOfImages))/(numberOfPixelsPerImage*noOfImages)
varianceB = (squaredSumOfBatchesB - ((sumOfBatchesB)**2)/(numberOfPixelsPerImage*noOfImages))/(numberOfPixelsPerImage*noOfImages)

#final std of R, G, B
stdR = math.sqrt(varianceR)
stdG = math.sqrt(varianceG)
stdB = math.sqrt(varianceB)

print(meanR)
print(meanG)
print(meanB)
print('---')
print(stdR)
print(stdG)
print(stdB)
  

0.006184743944666335
0.006139940288171414
0.006088251754062657
0.06234466895611421
0.061939846592852496
0.06151409079587224


In [ ]:
meanR = 0.006185
meanG = 0.006140
meanB = 0.006088
stdR = 0.06234
stdG = 0.06194
stdB = 0.06151

In [ ]:
#transformation for model training and testing
transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[meanR, meanG, meanB], std=[stdR, stdG, stdB])
])


#datasets and dataloaders for model training and testing
trainDataset = DDDdataset(train[:,0], train[:,-1], transformation)
testDataset = DDDdataset(test[:,0], test[:,-1], transformation)

trainDataLoader = DataLoader(trainDataset, batch_size=64, shuffle=True)
testDataLoader = DataLoader(testDataset, batch_size=64, shuffle=True)

In [ ]:
"""

--------------------------------
number of parameters CNN below

(II) f*f*ch_in*ch_out + ch_out_bias = params_conv_layer

(III) numberOfInputImageElements*outputFeaturesCurrentFCLayer = params_fully_connected

conv1: 4*4*3*10 + 10 = 490 -> 4*4 filter, 3 input image channels, 10 number of filters, +10 bias parameters of each filter
conv2: 5*5*10*20 + 20 = 5020
conv3: 6*6*20*32 + 32 = 23072
conv4: 3*3*32*64 + 64 = 18496

after conv4 layer: output "image": 3*3*64 = 576

fc1: 576*250 + 250 = 144250 -> fc1 has 250 output features
fc2: 250*50 + 50 = 12550
fc3: 50*2 + 2 = 102

complete CNN: 203980 parameters
"""

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(4,4), stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(10)
    self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(5,5), stride=2, padding=1)
    self.bn2 = nn.BatchNorm2d(20)
    self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=(6,6), stride=2, padding=1)
    self.bn3 = nn.BatchNorm2d(32)
    self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=1, padding=1)
    #self.bn4 = nn.BatchNorm2d(64)

    self.fc1 = nn.Linear(in_features=64*3*3, out_features=250)
    #self.bn5 = nn.BatchNorm1d(250)
    self.fc2 = nn.Linear(in_features=250, out_features=50)
    self.bn6 = nn.BatchNorm1d(50)
    self.fc3 = nn.Linear(in_features=50, out_features=2)

  def forward(self, X):

    X = F.relu(self.bn1(self.conv1(X)))

    X = F.max_pool2d(X, 2)

    X = F.relu(self.bn2(self.conv2(X)))

    X = F.max_pool2d(X, 2)

    X = F.relu(self.bn3(self.conv3(X)))

    X = F.max_pool2d(X, 3, stride=2)

    X = F.relu(self.bn4(self.conv4(X)))

    X = F.max_pool2d(X, 2)

    X = X.view(X.shape[0], -1)
    X = F.relu(self.bn5(self.fc1(X)))
    X = F.relu(self.bn6(self.fc2(X)))
    X = self.fc3(X)

    return X




In [ ]:
import time

model = CNN()
number_epochs = 2
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
losses = []
accuracies = []


#for the analysis of the wrong classifications
pathsOfWrongClassifications = []
rowsOfWrongClassifications = []
correctLabel = []
epochOfWrongClassification = []


start = time.time()

#model training and testing process
for epoch in range(number_epochs):

  #
  epoch_loss = 0
  epoch_accuracy = 0
  
  #-------model training-------
  for (X, y, path) in trainDataLoader:
    X = torch.as_tensor(X)
    y = torch.as_tensor(y)
    predictions = model(X)
    loss = loss_func(predictions, y.long())

    #adaption of the neural network
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #
    accuracy = ((predictions.argmax(dim=1) == y).float().mean())
    epoch_accuracy += accuracy
    epoch_loss += loss
  
  #total epoch_accuracy and epoch_loss calculation
  epoch_accuracy = epoch_accuracy/len(trainDataLoader)
  accuracies.append(epoch_accuracy)
  epoch_loss = epoch_loss / len(trainDataLoader)
  losses.append(epoch_loss)

  print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))
  #-------model training-------
  
  
  #---model testing---
  model.eval() #sets the model in evaluation mode
  
  #with disabled gradient calculation
  with torch.no_grad():
    test_epoch_loss = 0
    test_epoch_accuracy = 0

    for test_X, test_y, path in testDataLoader:

      test_preds = model(test_X)
      #print(test_preds.shape)
      #print(test_preds)
      test_loss = loss_func(test_preds, test_y.long())

      test_epoch_loss += test_loss            
      test_accuracy = ((test_preds.argmax(dim=1) == test_y).float().mean())
      test_epoch_accuracy += test_accuracy

      #if test prediction is wrong save the path of the image that has been classified wrong
      for index, row in enumerate(test_preds):
        if row.argmax(dim=0) != test_y[index]:
          pathsOfWrongClassifications.append(path[index])
          if rowsOfWrongClassifications == []:
            rowsOfWrongClassifications = row
          else:
            rowsOfWrongClassifications = np.vstack([rowsOfWrongClassifications, row])
          correctLabel.append(test_y[index].item())
          epochOfWrongClassification.append(epoch)

    #total test_epoch_accuracy and test_epoch_loss calculation
    test_epoch_accuracy = test_epoch_accuracy/len(testDataLoader)
    test_epoch_loss = test_epoch_loss / len(testDataLoader)

    print("Epoch: {}, test loss: {:.4f}, test acc: {:.4f}, time: {}\n".format(epoch, test_epoch_loss, test_epoch_accuracy, time.time() - start))
  #---model testing---

  model.train(mode=True) #sets the model back in training mode



 --- Epoch: 0, train loss: 0.0318, train acc: 0.9927, time: 5820.635276794434


<ipython-input-9-6d36089d14ce>:77: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if rowsOfWrongClassifications == []:


Epoch: 0, test loss: 0.3710, test acc: 0.8644, time: 7559.003975391388


 --- Epoch: 1, train loss: 0.0086, train acc: 0.9967, time: 8586.196023225784
Epoch: 1, test loss: 0.5604, test acc: 0.8842, time: 8766.524643421173



In [ ]:
#np_rows = np.array(rowsOfWrongClassifications)

#extract the person ID out of the stored image paths
personID = []
for pathStr in pathsOfWrongClassifications:
  personID.append(pathStr[pathStr.rfind('/')+1:-8])


#create a dataframe from all wrong classified images in the test dataset
dfWrong = pd.DataFrame()
dfWrong['testEpochNo'] = epochOfWrongClassification
dfWrong['personID'] = personID
dfWrong['imagepath'] = pathsOfWrongClassifications
dfWrong['correctLabel'] = correctLabel
dfWrong['outputValueLabel0'] = rowsOfWrongClassifications[:,0]
dfWrong['outputValueLabel1'] = rowsOfWrongClassifications[:,1]

print(dfWrong)

      testEpochNo personID                                          imagepath  \
0               0        E  /content/drive/MyDrive/newdata/Driver Drowsine...   
1               0        E  /content/drive/MyDrive/newdata/Driver Drowsine...   
2               0        E  /content/drive/MyDrive/newdata/Driver Drowsine...   
3               0        E  /content/drive/MyDrive/newdata/Driver Drowsine...   
4               0        G  /content/drive/MyDrive/newdata/Driver Drowsine...   
...           ...      ...                                                ...   
2614            1        e  /content/drive/MyDrive/newdata/Driver Drowsine...   
2615            1        e  /content/drive/MyDrive/newdata/Driver Drowsine...   
2616            1        G  /content/drive/MyDrive/newdata/Driver Drowsine...   
2617            1        e  /content/drive/MyDrive/newdata/Driver Drowsine...   
2618            1        e  /content/drive/MyDrive/newdata/Driver Drowsine...   

      correctLabel  outputV

In [ ]:
#get an overview of the persons that are likely to be classified in a wrong way
dfOfEpoch1 = dfWrong[dfWrong['testEpochNo']==1]
dfOfEpoch1.groupby('personID').size()

personID
E       5
G      48
b       7
c       1
e    1000
g      97
n      10
o       2
r       2
s      18
u       8
w       5
x       1
y       3
dtype: int64

Results of different trials:


I)

persons that have been shifted totally to the training dataset:
e, E, g, G

wrong classifications:

E: 5/962 = 0.5%

G: 48/499 = 9.6%

e: 1000/1000 = 100%

g: 97/109 = 89.0%

s: 18/457 = 3,9%

S: 0/487 = 0.0%



